# Set Up and Test Environment with OpenAI Gym

In [4]:
import gym
import random as r 

# Create Deep Learning Model
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

# Apply Reinforcement Learning
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [5]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
# This is for testing our environment
# Uncomment if you want to see how the game works

# episodes = 5

# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0

#     while not done:
#         env.render()
#         action = r.choice([0,1,2,3,4,5])
#         nState, reward, done, info = env.step(action)
#         score += reward
    
#     print(f'Episode: {episode} Score: {score}')

# env.close()

Episode: 1 Score: 20.0
Episode: 2 Score: 200.0
Episode: 3 Score: 75.0
Episode: 4 Score: 225.0
Episode: 5 Score: 75.0


# Create a Deep Learning Model with Keras and Tensorflow

In [2]:
def buildModel(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    
    return model

In [6]:
model = buildModel(height, width, channels, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

# Build Agent with Keras-RL

In [7]:
def buildAgent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.0, value_min=0.1, value_test=0.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return dqn

dqn = buildAgent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [8]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
  594/10000: episode: 1, duration: 20.577s, episode steps: 594, steps per second:  29, episode reward: 155.000, mean reward:  0.261 [ 0.000, 30.000], mean action: 2.451 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1088/10000: episode: 2, duration: 162.330s, episode steps: 494, steps per second:   3, episode reward: 65.000, mean reward:  0.132 [ 0.000, 20.000], mean action: 2.591 [0.000, 5.000],  loss: 22.628159, mean_q: 10.060919, mean_eps: 0.906040
 1939/10000: episode: 3, duration: 1278.813s, episode steps: 851, steps per second:   1, episode reward: 240.000, mean reward:  0.282 [ 0.000, 30.000], mean action: 2.501 [0.000, 5.000],  loss: 1.132383, mean_q: 7.937387, mean_eps: 0.863830
 2978/10000: episode: 4, duration: 1343.653s, episode steps: 1039, steps per second:   1, episode reward: 225.000, mean reward:  0.217 [ 0.000, 30.000], mean action: 2.540 [0.000, 5.000],  loss: 0.724818, mean_q: 7.831339, mean_eps: 0.778780
 3918/10000: episode: 5, d

In [9]:
# Run and test our AI
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 115.000, steps: 663
Episode 2: reward: 65.000, steps: 483
Episode 3: reward: 80.000, steps: 617
Episode 4: reward: 125.000, steps: 645
Episode 5: reward: 80.000, steps: 607
Episode 6: reward: 45.000, steps: 402
Episode 7: reward: 315.000, steps: 905
Episode 8: reward: 455.000, steps: 805
Episode 9: reward: 170.000, steps: 661
Episode 10: reward: 210.000, steps: 994
166.0
